In [1]:
from platform import python_version
import numpy as np
import pandas as pd
np.random.seed(42) # set the seed to make examples repeatable

In [2]:
size = 10000
cities = ["paris", "barcelona", "berlin", "new york"]
df = pd.DataFrame(
    {"city": np.random.choice(cities, size=size), "booked_perc": np.random.rand(size)}
)
df["id"] = df.index.map(str) + "-" + df.city
df = df[["id", "city", "booked_perc"]]


In [3]:
df.head(2)

,id,city,booked_perc
0,0-berlin,berlin,0.393636
1,1-new york,new york,0.473436


# 1. How NOT to sum the data


In [8]:
%%timeit
suma = 0
for _, row in df.iterrows():
    suma += row.booked_perc
    
##766 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.23 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
##pythonic way to sum:
sum(booked_perc for booked_perc in df.booked_perc) 

4964.310889430854

# 2. How NOT to filter the data

In [24]:
##%%timeit
suma = 0
for _, row in df.iterrows():
    if row.booked_perc <= 0.5:
        suma += row.booked_perc
##831 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
##%%timeit

##724 µs ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

In [14]:
df[(df.booked_perc <= 0.5) & (df.city == 'new york')].booked_perc.sum()
#PYTHONIC WAY TO SUM

313.49350304505055

# 3. How NOT to access previous values

In [26]:
for i in range(1, len(df)):
    df.loc[i, "perc_change"] =  (df.loc[i].booked_perc - df.loc[i - 1].booked_perc) / df.loc[i - 1].booked_perc
##7.02 s ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
##%%timeit
##AGAIN THE SECOND EXAMPLE IS WAY FASTER AND MORE PYTHONIC

df["perc_change"] = df.booked_perc.pct_change()
#586 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

# 4. How NOT to apply complex functions

In [15]:
##The first approach that comes to my mind is using a for loop with iterrows.
##%%timeit
for i, row in df.iterrows():
    if row.city == 'new york':
        df.loc[i, 'sales_factor'] = row.booked_perc * 2
    else:
        df.loc[i, 'sales_factor'] = 0
##3.58 s ± 48.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [16]:
##A slightly better approach is using apply function directly on a DataFrame.
##%%timeit
def calculate_sales_factor(row):
    if row.city == 'new york':
        return row.booked_perc * 2
    return 0
df['sales_factor'] = df.apply(calculate_sales_factor, axis=1)
##165 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [18]:
##The fastest way is to use pandas filter and calculate the function values directly.
##%%timeit 
df.loc[df.city == 'new york', 'sales_factor'] = df[df.city == 'new york'].booked_perc * 2
df.sales_factor.fillna(0, inplace=True)
##3.03 ms ± 85.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

# 5. How NOT to group data


In [19]:
##Let’s say we would like to calculate:
##the average sales factor by the city
#3and the first booking id by the city.
#%%timeit 
avg_by_city = {}
count_by_city = {}
first_booking_by_city = {}
for i, row in df.iterrows():
    city = row.city
    if city in avg_by_city:
        avg_by_city[city] += row.sales_factor
        count_by_city[city] += 1
    else:
        avg_by_city[city] = row.sales_factor
        count_by_city[city] = 1
        first_booking_by_city[city] = row['id']
for city, _ in avg_by_city.items():
    avg_by_city[city] /= count_by_city[city]
#878 ms ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [20]:
#Pandas has a group by operation so there is no need to iterate over a DataFrame. group by in pandas does the same thing as GROUP BY statement in SQL.
#%%timeit
df.groupby('city').sales_factor.mean()
df.groupby('city').sales_factor.count()
df.groupby('city').id.first()
#3.05 ms ± 65.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
#%%timeit
df.groupby("city").agg({"sales_factor": ["mean", "count"], "id": "first"})
#4.5 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

sales_factor                 id
                  mean count        first
city                                     
barcelona     0.000000  2486  9-barcelona
berlin        0.000000  2477     0-berlin
new york      0.998901  2501   1-new york
paris         0.000000  2536      2-paris

In [22]:
df.shape

(10000, 4)